In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
loans = pd.read_csv('data/lending-club-data.csv')
train_idx = pd.read_json('data/module-5-assignment-2-train-idx.json')[0]
test_idx = pd.read_json('data/module-5-assignment-2-test-idx.json')[0]
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.drop(columns=['bad_loans'])
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]

target = 'safe_loans'                    # prediction target (y) (+1 means safe, -1 is risky)

# Extract the feature columns and target column
loans = loans[features + [target]]
train_data = loans.iloc[train_idx]
test_data = loans.iloc[test_idx]

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def one_hot_encoding(df, target):
    categorical_features = []
    for feature_name, feature_type in zip(df.columns, df.dtypes):
        if feature_type is np.dtype('O'):
            categorical_features.append(feature_name)
            
    for feature_name in categorical_features:
        df2 = pd.get_dummies(df[feature_name],prefix=feature_name)
        df = df.join(df2).drop([feature_name], axis=1)

    t = df[target]
    return (df.drop([target], axis=1), t)


train_data, train_target = one_hot_encoding(train_data, target)
test_data, test_target = one_hot_encoding(test_data, target)

In [113]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0 
    
    a = (labels_in_node == 1).sum()
    b = (labels_in_node == -1).sum()
    return min(a,b)


def best_splitting_feature(data, features, target_values):
    
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        left_split = data[data[feature] == 0]
        
        right_split = data[data[feature] == 1]
            
        left_mistakes = intermediate_node_num_mistakes(target_values[data[feature] == 0])
        right_mistakes = intermediate_node_num_mistakes(target_values[data[feature]==1])
            
        error = (left_mistakes+right_mistakes)/len(target_values)
        if error < best_error:
            best_error = error
            best_feature = feature
    
    return best_feature


def create_leaf(target_values):    
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True}
   
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    #set the prediction to be the majority class.
    if num_ones > num_minus_ones:
        leaf['prediction'] = 1
    else:
        leaf['prediction'] = -1
    return leaf 


def decision_tree_create(data, features, target_values, current_depth = 0, max_depth = 10):
    remaining_features = features[:] 
    
    print ("--------------------------------------------------------------------")
    print ("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    

    if intermediate_node_num_mistakes(target_values) == 0:
        print ("Stopping condition 1 reached.")     
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if len(remaining_features) == 0:
        print("Stopping condition 2 reached.")    
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth:
        print("Reached maximum depth. Stopping for now.")
        return create_leaf(target_values)

    splitting_feature = best_splitting_feature(data, remaining_features, target_values)

    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    left_target = target_values[data[splitting_feature] == 0]
    right_target = target_values[data[splitting_feature] == 1]
    remaining_features = remaining_features.drop(splitting_feature)
    print ("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(left_target)
    if len(right_split) == len(data):
        print ("Creating right node.")
        return create_leaf(right_target)
        
    left_tree = decision_tree_create(left_split, remaining_features, left_target, current_depth + 1, max_depth)        
    right_tree = decision_tree_create(right_split, remaining_features, right_target, current_depth + 1, max_depth)

    return {'is_leaf'          : False,
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}


def classify(tree, x, annotate = False):
#     classify single node x with dicision tree 
       # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
             print ("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print ("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return (classify(tree['left'], x, annotate))
        else:
            return (classify(tree['right'], x, annotate))
        

def evaluate_classification_error(tree, data, target):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x), axis=1)
    return (prediction != target).sum()/len(target)


def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print ("(leaf, label: %s)" % tree['prediction'])
        return None
    split_feature, split_value = split_name.split('_')
    print ('                       %s' % name)
    print ('         |---------------|----------------|')
    print ('         |                                |')
    print ('         |                                |')
    print ('         |                                |')
    print ('  [{0} == 0]               [{0} == 1]    '.format(split_name))
    print ('         |                                |')
    print ('         |                                |')
    print ('         |                                |')
    print ('    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree')))
    

def reached_minimum_node_size(data, min_node_size):
    return True if data.shape[0] <= min_node_size else False
    

def error_reduction(error_before_split, error_after_split):
    return error_before_split-error_after_split


def decision_tree_create_EarlyStop(data, features, target_values, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] 
    
    print ("--------------------------------------------------------------------")
    print ("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    

    if intermediate_node_num_mistakes(target_values) == 0:
        print ("Stopping condition 1 reached. All data points have the same target value.")     
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if len(remaining_features) == 0:
        print("Stopping condition 2 reached. No remaining features.")    
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth:
        print("Early stopping condition 1 reached. Reached maximum depth.")
        return create_leaf(target_values)
    
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if  reached_minimum_node_size(data, min_node_size):
        print ("Early stopping condition 2 reached. Reached minimum node size.")
        return create_leaf(target_values)

    splitting_feature = best_splitting_feature(data, remaining_features, target_values)

    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    left_target = target_values[data[splitting_feature] == 0]
    right_target = target_values[data[splitting_feature] == 1]
    
    # Early stopping condition 3: Minimum error reduction
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    left_mistakes = intermediate_node_num_mistakes(left_target)
    right_mistakes = intermediate_node_num_mistakes(right_target)
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    if  error_reduction(error_before_split, error_after_split) <= min_error_reduction:
        print ("Early stopping condition 3 reached. Minimum error reduction.")
        return create_leaf(target_values)
    
    remaining_features = remaining_features.drop(splitting_feature)
    print ("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(left_target)
    if len(right_split) == len(data):
        print ("Creating right node.")
        return create_leaf(right_target)
        
    left_tree = decision_tree_create_EarlyStop(left_split, remaining_features, left_target, current_depth + 1, 
                                               max_depth, min_node_size, min_error_reduction)        
    right_tree = decision_tree_create_EarlyStop(right_split, remaining_features, right_target, current_depth + 1, 
                                                max_depth, min_node_size, min_error_reduction)

    return {'is_leaf'          : False,
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}


def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])



In [109]:
a = pd.DataFrame([[1,2,3,5],[4,5,6],[7,8,9]])
a.shape
True if False else False

False

In [26]:
my_decision_tree = decision_tree_create(train_data, train_data.columns, train_target, current_depth = 0, max_depth = 6)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
R

In [29]:
print(test_data.loc[24])
print('Predicted class: %s ' % classify(my_decision_tree, test_data.loc[24], annotate=True))

grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    1
grade_E                    0
grade_F                    0
grade_G                    0
term_ 36 months            0
term_ 60 months            1
home_ownership_MORTGAGE    0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        1
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         1
emp_length_3 years         0
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
Name: 24, dtype: uint8
Split on term_ 36 months = 0
Split on grade_A = 0
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
At leaf, predicting -1
Predicted class: -1 


In [36]:
evaluate_classification_error(my_decision_tree, test_data, test_target)

0.6222533390779836

In [43]:
print_stump(my_decision_tree)
print_stump(my_decision_tree['left'], my_decision_tree['splitting_feature'])
print_stump(my_decision_tree['left']['left'], my_decision_tree['left']['splitting_feature'])

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term_ 36 months == 0]               [term_ 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)
                       term_ 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_A == 0]               [grade_A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)
                       grade_A
         |---------------|----------------|
         |                    

In [45]:
print(my_decision_tree['splitting_feature'])
print(my_decision_tree['right']['splitting_feature'])
print(my_decision_tree['right']['right']['splitting_feature'])

term_ 36 months
grade_D
None


## Decision Trees in Practice:


In [103]:
train_idx = pd.read_json('data/module-6-assignment-train-idx.json')[0]
validation_idx = pd.read_json('data/module-6-assignment-validation-idx.json')[0]
train_data = loans.iloc[train_idx]
validation_data = loans.iloc[validation_idx]
train_data, train_target = one_hot_encoding(train_data, target)
validation_data, validation_target = one_hot_encoding(validation_data, target)

In [104]:
my_decision_tree_new = decision_tree_create_EarlyStop(train_data, train_data.columns, train_target, current_depth = 0, 
                         max_depth = 6, min_node_size=100, 
                         min_error_reduction=0.0)
my_decision_tree_old = decision_tree_create_EarlyStop(train_data, train_data.columns, train_target, current_depth = 0, 
                         max_depth = 6, min_node_size=0, 
                         min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature emp_length_< 1 year. (90, 11)
--------------------------------------------------------------------
Subtree, depth = 3 (90 data points).
Early stopping condition 2 reached. Reached minimum node size.
--------------------------------------------------------------------
Subtree, depth = 3 (11 data points).
Early stopping condition 2 reached. Reached minimum node size.
------------------------------------

In [107]:
print(count_leaves(my_decision_tree_new),
count_leaves(my_decision_tree_old))

11 18


In [105]:
# print(validation_data.loc[24])
print('Predicted class: %s \n' % classify(my_decision_tree_new, validation_data.loc[24], annotate=True))
print('Predicted class: %s ' % classify(my_decision_tree_old, validation_data.loc[24], annotate=True))

Split on term_ 36 months = 0
Split on grade_A = 0
At leaf, predicting -1
Predicted class: -1 

Split on term_ 36 months = 0
Split on grade_A = 0
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
At leaf, predicting -1
Predicted class: -1 


In [114]:
print(evaluate_classification_error(my_decision_tree_new, validation_data, validation_target))
print(evaluate_classification_error(my_decision_tree, validation_data, validation_target))

0.37774666092201636
0.37774666092201636


In [91]:
model_depth2 = decision_tree_create_EarlyStop(train_data, train_data.columns, train_target, current_depth = 0, 
                         max_depth = 2, min_node_size=0, 
                         min_error_reduction=-1)
model_depth6 = decision_tree_create_EarlyStop(train_data, train_data.columns, train_target, current_depth = 0, 
                         max_depth = 6, min_node_size=0, 
                         min_error_reduction=-1)
model_depth14 = decision_tree_create_EarlyStop(train_data, train_data.columns, train_target, current_depth = 0, 
                         max_depth = 14, min_node_size=0, 
                         min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
Split on feature grade_D. (23300, 4701)
--------------------------------------------------------------------
Subtree, depth = 2 (23300 data points).
Early stopping condition 1 reached. Reached maximum depth.
-----------------------------------------------

Split on feature grade_E. (2058, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 4 (2190 data points).
Split on feature grade_D. (2190, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 3 (1048 data points).
Split on feature emp_length_5 years. (969, 79)
--------------------------------------------------------------------
Subtree, depth = 4 (969 data points).
Split on feature grade_C. (969, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 4 (79 data points).
Split on feature home_ownership_MORTGAGE. (34, 45)
--------------------------------------------------------------------
Subtree, depth = 5 (34 data points).
Split on feature grade_C. (34, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 5 (45 data points).
Split on feature grade_C. (45, 0)
Creati

Split on feature grade_A. (4701, 0)
Creating leaf node.


In [115]:
print(evaluate_classification_error(model_depth2, train_data, train_target))
print(evaluate_classification_error(model_depth6, train_data, train_target))
print(evaluate_classification_error(model_depth14, train_data, train_target))
print()
print(evaluate_classification_error(model_depth2, validation_data, validation_target))
print(evaluate_classification_error(model_depth6, validation_data, validation_target))
print(evaluate_classification_error(model_depth14, validation_data, validation_target))

0.40003761014399314
0.3804266064904363
0.37892220073071137

0.3981042654028436
0.37774666092201636
0.37882378285221885


In [93]:
print(count_leaves(model_depth2),
count_leaves(model_depth6),
count_leaves(model_depth14))

4 18 35


In [94]:
model_minErrorReduction_ignore = decision_tree_create_EarlyStop(train_data, train_data.columns, train_target, current_depth = 0, 
                         max_depth = 6, min_node_size=0, 
                         min_error_reduction=-1)
model_minErrorReduction_0 = decision_tree_create_EarlyStop(train_data, train_data.columns, train_target, current_depth = 0, 
                         max_depth = 6, min_node_size=0, 
                         min_error_reduction=0)
model_minErrorReduction_5 = decision_tree_create_EarlyStop(train_data, train_data.columns, train_target, current_depth = 0, 
                         max_depth = 6, min_node_size=0, 
                         min_error_reduction=5)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
E

Early stopping condition 3 reached. Minimum error reduction.


In [116]:
print(evaluate_classification_error(model_minErrorReduction_ignore, validation_data, validation_target))
print(evaluate_classification_error(model_minErrorReduction_0, validation_data, validation_target))
print(evaluate_classification_error(model_minErrorReduction_5, validation_data, validation_target))

0.37774666092201636
0.37774666092201636
0.503446790176648


In [96]:
print(count_leaves(model_minErrorReduction_ignore),
count_leaves(model_minErrorReduction_0),
count_leaves(model_minErrorReduction_5))

18 12 1


In [97]:
model_minNodeSize_0 = decision_tree_create_EarlyStop(train_data, train_data.columns, train_target, current_depth = 0, 
                         max_depth = 6, min_node_size=0, 
                         min_error_reduction=-1)
model_minNodeSize_2000 = decision_tree_create_EarlyStop(train_data, train_data.columns, train_target, current_depth = 0, 
                         max_depth = 6, min_node_size=2000, 
                         min_error_reduction=-1)
model_minNodeSize_50000 = decision_tree_create_EarlyStop(train_data, train_data.columns, train_target, current_depth = 0, 
                         max_depth = 6, min_node_size=50000, 
                         min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
E

In [117]:
print(evaluate_classification_error(model_minNodeSize_0, validation_data, validation_target))
print(evaluate_classification_error(model_minNodeSize_2000, validation_data, validation_target))
print(evaluate_classification_error(model_minNodeSize_50000, validation_data, validation_target))

0.37774666092201636
0.3774235243429556
0.503446790176648


In [99]:
print(count_leaves(model_minNodeSize_0),
count_leaves(model_minNodeSize_2000),
count_leaves(model_minNodeSize_50000))

18 13 1
